# Combine the weatherstation and fix problems

In [1]:
import numpy as np 
import pandas as pd
from os import listdir
from os.path import isfile, join, isdir
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pickle
import re
import math
from datetime import datetime, timedelta
from tests import *
from tools import *
import numpy_indexed as npi

# Idea: Plotcode /Date should be the unique keypair. Need some transforms

In [5]:
#vars 
path = "../Canbus_Data/covariables"

In [6]:
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

In [7]:
cont = []
for file in onlyfiles: 
    if "soil respiration" in file:
        cont.append(pd.read_csv(path + "/" + file,sep=";"))
    else: 
        cont.append(pd.read_csv(path + "/" + file,sep=","))

In [420]:
# plot 1

In [10]:
cont[0]

,year,season,plotcode,SRweed,group,species_other,cover
0,2020,May,B4A20,16.0,Target,Rum.ace,2.0
1,2020,May,B4A20,16.0,Target,Vic.cra,0.0
2,2020,May,B4A20,16.0,Target,Tra.pra,1.0
3,2020,May,B4A20,16.0,Target,Tri.fra,0.0
4,2020,May,B4A21,22.0,Target,Dau.car,0.0
...,...,...,...,...,...,...,...
36927,2003,May,B4A22,NaN,Plot.level,Bareground,NaN
36928,2003,May,B4A22,NaN,Target,Cam.pat,2.0
36929,2003,May,B4A22,NaN,Target,Car.pra,1.0
36930,2003,May,B4A22,NaN,Target,Ger.pra,2.0


In [421]:
times = []
for x in range(len(cont[0])):
    month = cont[0]["season"][x]
    year = cont[0]["year"][x]
    times.append(pd.Timestamp("15." + str(month) + "." + str(year) ))

In [422]:
cont[0].drop(["season","year"],axis = 1, inplace = True)
cont[0]["time"] = times
cont[0].rename(columns={'plotcode': 'plot'}, inplace = True)

In [423]:
# plot 2

In [424]:
times = []
for x in range(len(cont[1])):
    month = cont[1]["season"][x]
    year = cont[1]["year"][x]
    times.append(pd.Timestamp("15." + str(month) + "." + str(year) ))

In [425]:
cont[1].drop(["season","year"],axis= 1, inplace = True)
cont[1]["time"] = times
cont[1].rename(columns={'plotcode': 'plot'}, inplace = True)

In [426]:
# plot 3

In [427]:
set(list(cont[2]["season"].values))

{'Aug', 'May', 'annual'}

In [428]:
times = []
for x in range(len(cont[2])):
    month = cont[2]["season"][x]
    year = cont[2]["year"][x]
    if month == "annual":
        times.append(pd.Timestamp("01." + str("July") + "." + str(year) ))
    else:
        times.append(pd.Timestamp("15." + str(month) + "." + str(year) ))

In [429]:
cont[2].drop(["season","year"],axis= 1, inplace = True)
cont[2]["time"] = times
cont[2].rename(columns={'plotcode': 'plot'}, inplace = True)

In [430]:
# plot 4

In [431]:
times = []
for x in range(len(cont[3])):
    year = cont[3]["year"][x]
    times.append(pd.Timestamp("01." + str("July") + "." + str(year) ))

In [432]:
cont[3].drop(["year"],axis= 1, inplace = True)
cont[3]["time"] = times

In [433]:
# plot 5

In [434]:
times = []
for x in range(len(cont[4])):
    month = cont[4]["month"][x]
    year = cont[4]["year"][x]
    if month < 10: 
        times.append(pd.Timestamp("15.0" + str(month) + "." + str(year) ))
    else:
        times.append(pd.Timestamp("15." + str(month) + "." + str(year) ))

In [435]:
cont[4].drop(["Unnamed: 0","month","year"],axis = 1, inplace = True)
cont[4]["time"] = times

In [436]:
cont[4]["time"] = times

In [437]:
# plot 6

In [438]:
times = []
for x in range(len(cont[5])):
    month = cont[5]["season"][x]
    year = cont[5]["year"][x]
    times.append(pd.Timestamp("15." + str(month) + "." + str(year) ))

In [439]:
cont[5].drop(["season","year"],axis= 1, inplace = True)
cont[5]["time"] = times
cont[5].rename(columns={'plotcode': 'plot'}, inplace = True)

In [440]:
# table 0  
# table 1 good! 
# table 2
# table 3 just doubles? kick out doubles
# table 4 depth.min/max in multiple levels
# table 5 10 subsamples per measurement --> make values for every subsample 

### table 1


In [441]:
cols = []
for x in list(set(cont[0]["species_other"])):
    cols.append(cont[0][cont[0]["species_other"] == x].loc[:,["cover","group","SRweed", "time", "plot"]].rename(columns={"cover": "cover_" + str(x),
                                                                                                                      "group": "group_"+ str(x),
                                                                                                                        "SRweed": "SRweed_"+ str(x)}))

In [442]:
out = cols[0].merge(cols[1], on=["time","plot"], how="outer")
for x in range(2,len(cols)):
    out = out.merge(cols[x], on=["time","plot"], how="outer")

In [443]:
count = 0 
for x in list(set(cont[0]["time"])):
    for y in list(set(cont[0]["plot"])):
        if len(cont[0][(cont[0]["plot"] == y) & (cont[0]["time"] == x)]) ==  0: 
            count += 1

In [444]:
count # test. merge works correctly

44

In [445]:
cont[0] = out

### table 2

In [446]:
for x in list(set(cont[1]["plot"])):
    for y in list(set(cont[1]["time"])):
        if len(cont[1][(cont[1]["plot"] == x) & (cont[1]["time"] == y)]) > 1: 
                  print(x,y, len(cont[1][(cont[1]["plot"] == x) & (cont[1]["time"] == y)]))
## All unique. No problems here

### table 3

In [447]:
cols = []
for x in list(set(cont[2]["species_other"])):
    for y in list(set(cont[2]["subsample"])):
        cols.append(cont[2][(cont[2]["species_other"] == x) & (cont[2]["subsample"] == y)].loc[:,["biomass", "time", "plot"]].rename(columns={
                                                                                                                        "biomass": "biomass_" + str(x) + "_sample_" + str(y)}))

In [448]:
out = cols[0].merge(cols[1], on=["time","plot"], how="outer")
for x in range(2,len(cols)):
    out = out.merge(cols[x], on=["time","plot"], how="outer")

In [449]:
cont[2] = out

### table 4

In [450]:
for x in list(set(cont[3]["plot"])):
    for y in list(set(cont[3]["time"])):
        if len(cont[3][(cont[3]["plot"] == x) & (cont[3]["time"] == y)]) > 1: 
                  print(x,y, len(cont[3][(cont[3]["plot"] == x) & (cont[3]["time"] == y)]))

B2A17 2018-07-01 00:00:00 2
B3A01 2008-07-01 00:00:00 2
B3A08 2019-07-01 00:00:00 2
B4A17 2019-07-01 00:00:00 2
B4A17 2018-07-01 00:00:00 2
B1A14 2020-07-01 00:00:00 2
B3A09 2019-07-01 00:00:00 2
B1A04 2018-07-01 00:00:00 2
B4A07 2020-07-01 00:00:00 2
B1A20 2018-07-01 00:00:00 2
B1A22 2020-07-01 00:00:00 2
B4A02 2020-07-01 00:00:00 2


In [451]:
cont[3].drop_duplicates(subset=["time","plot"], inplace=True)
#12 duplicates. kickout and done.

### table 5

In [452]:
for x in list(set(cont[4]["plot"])):
    for y in list(set(cont[4]["time"])):
        if len(cont[4][(cont[4]["plot"] == x) & (cont[4]["time"] == y)]) > 1: 
                  print(x,y, len(cont[4][(cont[4]["plot"] == x) & (cont[4]["time"] == y)]))

B1A05 2017-06-15 00:00:00 5
B1A05 2014-06-15 00:00:00 5
B1A05 2011-06-15 00:00:00 5
B1A05 2008-06-15 00:00:00 4
B3A14 2017-06-15 00:00:00 5
B3A14 2014-06-15 00:00:00 5
B3A14 2011-06-15 00:00:00 5
B3A14 2008-06-15 00:00:00 4
B3A03 2017-06-15 00:00:00 5
B3A03 2014-06-15 00:00:00 5
B3A03 2011-06-15 00:00:00 5
B3A03 2008-06-15 00:00:00 4
B2A09 2017-06-15 00:00:00 5
B2A09 2014-06-15 00:00:00 5
B2A09 2011-06-15 00:00:00 5
B2A09 2008-06-15 00:00:00 4
B1A04 2017-06-15 00:00:00 5
B1A04 2014-06-15 00:00:00 5
B1A04 2011-06-15 00:00:00 5
B1A04 2008-06-15 00:00:00 4
B2A10 2017-06-15 00:00:00 5
B2A10 2014-06-15 00:00:00 5
B2A10 2011-06-15 00:00:00 5
B2A10 2008-06-15 00:00:00 4
B2A17 2017-06-15 00:00:00 5
B2A17 2014-06-15 00:00:00 5
B2A17 2011-06-15 00:00:00 5
B2A17 2008-06-15 00:00:00 4
B3A04 2017-06-15 00:00:00 5
B3A04 2014-06-15 00:00:00 5
B3A04 2011-06-15 00:00:00 5
B3A04 2008-06-15 00:00:00 4
B1A17 2017-06-15 00:00:00 5
B1A17 2014-06-15 00:00:00 5
B1A17 2011-06-15 00:00:00 5
B1A17 2008-06-15 00:

In [453]:
cont[4].loc[:,["depth.min", "depth.max"]].drop_duplicates()

,depth.min,depth.max
0,0.00,0.30
246,0.00,0.05
247,0.05,0.10
248,0.10,0.20
249,0.20,0.30
578,0.30,0.40


In [454]:
cols = []
for x in cont[4].loc[:,["depth.min", "depth.max"]].drop_duplicates().values:
        cols.append(cont[4][(cont[4]["depth.min"] == x[0]) & (cont[4]["depth.max"] == x[1])].loc[:,["root.bm","coarse_root.bm", "fine_root.bm", "time", "plot"]].rename(columns={
                                                                                                                        "root.bm": "root.bm_" + str(x[0]) + "-" + str(x[1]),
                                                                                                                        "coarse_root.bm": "coarse_root.bm_" + str(x[0]) + "-" + str(x[1]),
                                                                                                                        "fine_root.bm": "fine_root.bm_" + str(x[0]) + "-" + str(x[1])        
                                                                                                                                }))

In [455]:
out = cols[0].merge(cols[1], on=["time","plot"], how="outer")
for x in range(2,len(cols)):
    out = out.merge(cols[x], on=["time","plot"], how="outer")

In [456]:
cont[4] = out

### table 6

In [457]:
cols = []
for x in list(set(cont[5]["subsample"])):
    cols.append(cont[5][cont[5]["subsample"] == x].loc[:,["height_target_leaf","height_target_flower","time", "plot"]].rename(columns={"height_target_leaf": "height_target_leaf_sample_" + str(x),
                                                                                                                      "height_target_flower": "height_target_flower_sample_"+ str(x)}))

In [458]:
out = cols[0].merge(cols[1], on=["time","plot"], how="outer")
for x in range(2,len(cols)):
    out = out.merge(cols[x], on=["time","plot"], how="outer")

In [459]:
cont[5] = out

# Final merge

In [460]:
out = cont[0].merge(cont[1], on=["time","plot"], how="outer")
for x in range(2,len(cont)):
    out = out.merge(cont[x], on=["time","plot"], how="outer")

In [461]:
for x in list(set(out["plot"])):
    for y in list(set(out["time"])):
        if len(out[(out["plot"] == x) & (out["time"] == y)]) > 1: 
                  print(x,y, len(out[(out["plot"] == x) & (out["time"] == y)]))
#Worked. No double keys

In [462]:
cols = out.columns.tolist()
cols = [cols[3]] + [cols[4]] + cols[:3] + cols[5:]
out = out[cols]

In [463]:
out = out[cols]

In [464]:
pickle.dump(out, open("data_transform/covariates.p", "wb"))   